In [26]:
%load_ext autoreload
%autoreload 2

In [27]:
from collections import defaultdict
import math
import numpy as np
import pandas as pd
import pyalex
from pyalex import Authors, Concepts, Institutions, Works

from researcher_impact.pyalex_utils import *

In [28]:
# The polite pool has much faster and more consistent response times. To get into the polite pool, you set your email:
pyalex.config.email = "ben@epochai.org"

# PCD database

In [2]:
# Download dataset from the Parameters, Compute and Data Trends in ML sheet
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1AAIebjNsnJj_uKALHbXNfn3_YsT6sHXtCU0q7OIPuc4/export?format=csv#gid=0')

In [3]:
df

,System,Domain,Task,Organization,Organization Categorization,Authors,Publication date,Reference,Link,Citations,...,Training dataset size (GB),Approach,Training compute cost (2020 USD),Compute cost notes,Self-supervised training,Architecture,Compute Sponsor Categorization,Epistemic status,Abstract,Last Modified
0,PaLM 2,Language,Language modelling,Google Research,Industry,"Andrew M. Dai, David R. So, Dmitry Lepikhin, J...",2023-05-10,PaLM 2 Technical Report,https://ai.google/static/documents/palm2techre...,NaN,...,NaN,NaN,NaN,PaLM 2 was trained on TPU v4 according to the ...,NaN,NaN,Industry,NaN,"We introduce PaLM 2, a new state-of-the-art la...",2023-06-06 18:47:36
1,GPT-4,Multimodal,Language modelling,OpenAI,Industry,OpenAI,2023-03-15,GPT-4 Technical Report,https://arxiv.org/abs/2303.08774,NaN,...,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,2023-05-29 20:51:04
2,Phenaki,Vision,Video generation,"Google Brain, University College London, Unive...",Industry - Academia Collaboration (Industry le...,"Ruben Villegas, Mohammad Babaeizadeh, Pieter-J...",2022-10-05,Phenaki: Variable Length Video Generation From...,https://arxiv.org/abs/2210.02399,NaN,...,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,2023-05-29 20:51:04
3,Minerva (540B),Language,Quantitative Reasoning Problems,Google Research,Industry,"Aitor Lewkowycz, Anders Andreassen, David Doha...",2022-06-29,Solving Quantitative Reasoning Problems with L...,https://arxiv.org/abs/2206.14858,NaN,...,NaN,NaN,"$3,267,257.75",NaN,Yes,NaN,Industry,NaN,Language models have achieved remarkable perfo...,2023-06-08 00:39:43
4,PaLM (540B),Language,Language modelling,Google Research,Industry,"Aakanksha Chowdhery, Sharan Narang, Jacob Devl...",2022-04-04,PaLM: Scaling Language Modeling with Pathways,https://arxiv.org/abs/2204.02311,228.0,...,NaN,NaN,"$3,232,806.53",NaN,Yes,NaN,Industry,NaN,Large language models have been shown to achie...,2023-05-29 20:51:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,AR-LDM,Multimodal,Text-to-image,Alibaba,Industry - Academia Collaboration (Industry le...,"Xichen Pan, Pengda Qin, Yuhong Li, Hui Xue, We...",2022-11-20,Synthesizing Coherent Story with Auto-Regressi...,https://arxiv.org/abs/2211.10950,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Likely,Conditioned diffusion models have demonstrated...,2023-06-09 16:04:47
546,NaN,Vision,Image classification,"University of Guelph,Canadian Institute for Ad...",Industry - Academia Collaboration,"Terrance DeVries, Graham W. Taylor",2017-08-15,Improved Regularization of Convolutional Neura...,https://arxiv.org/abs/1708.04552,1450.0,...,NaN,NaN,NaN,https://www.yuzeh.com/data/agz-cost.html,NaN,NaN,Industry,NaN,NaN,2023-06-09 16:00:52
547,AltCLIP,Multimodal,NaN,BAAI,Academia,"Zhongzhi Chen, Guang Liu, Bo-Wen Zhang, Fulong...",2022-11-12,AltCLIP: Altering the Language Encoder in CLIP...,https://arxiv.org/abs/2211.06679,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Likely,"In this work, we present a conceptually simple...",2023-06-09 16:04:45
548,ALM 1.0,Language,Language modelling,BAAI,Academia,NaN,NaN,ALM 1.0,https://github.com/FlagAI-Open/FlagAI/blob/mas...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Speculative,NaN,2023-06-09 16:06:43


In [4]:
notable_df = df.dropna(subset=['Inclusion criteria'])
notable_df

,System,Domain,Task,Organization,Organization Categorization,Authors,Publication date,Reference,Link,Citations,...,Training dataset size (GB),Approach,Training compute cost (2020 USD),Compute cost notes,Self-supervised training,Architecture,Compute Sponsor Categorization,Epistemic status,Abstract,Last Modified
0,PaLM 2,Language,Language modelling,Google Research,Industry,"Andrew M. Dai, David R. So, Dmitry Lepikhin, J...",2023-05-10,PaLM 2 Technical Report,https://ai.google/static/documents/palm2techre...,NaN,...,NaN,NaN,NaN,PaLM 2 was trained on TPU v4 according to the ...,NaN,NaN,Industry,NaN,"We introduce PaLM 2, a new state-of-the-art la...",2023-06-06 18:47:36
1,GPT-4,Multimodal,Language modelling,OpenAI,Industry,OpenAI,2023-03-15,GPT-4 Technical Report,https://arxiv.org/abs/2303.08774,NaN,...,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,2023-05-29 20:51:04
3,Minerva (540B),Language,Quantitative Reasoning Problems,Google Research,Industry,"Aitor Lewkowycz, Anders Andreassen, David Doha...",2022-06-29,Solving Quantitative Reasoning Problems with L...,https://arxiv.org/abs/2206.14858,NaN,...,NaN,NaN,"$3,267,257.75",NaN,Yes,NaN,Industry,NaN,Language models have achieved remarkable perfo...,2023-06-08 00:39:43
4,PaLM (540B),Language,Language modelling,Google Research,Industry,"Aakanksha Chowdhery, Sharan Narang, Jacob Devl...",2022-04-04,PaLM: Scaling Language Modeling with Pathways,https://arxiv.org/abs/2204.02311,228.0,...,NaN,NaN,"$3,232,806.53",NaN,Yes,NaN,Industry,NaN,Large language models have been shown to achie...,2023-05-29 20:51:04
6,Chinchilla,Language,Language modelling,DeepMind,Industry,"Jordan Hoffmann, Sebastian Borgeaud, Arthur Me...",2022-03-29,Training Compute-Optimal Large Language Models,https://arxiv.org/abs/2203.15556,NaN,...,NaN,NaN,"$753,491.58",NaN,Yes,NaN,Industry,NaN,We investigate the optimal model size and numb...,2023-05-29 20:51:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,Taiyi-Stable\nDiffusion,Drawing,Text-to-image,NaN,NaN,NaN,2022-10-31,NaN,https://huggingface.co/IDEA-CCNL/Taiyi-Stable-...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Likely,NaN,2023-06-09 16:04:42
543,CogVideo,Multimodal,Video generation,"Tsinghua University,BAAI",Academia,"Wenyi Hong, Ming Ding, Wendi Zheng, Xinghan Li...",2022-05-29,CogVideo: Large-scale Pretraining for Text-to-...,https://arxiv.org/abs/2205.15868,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Likely,Large-scale pretrained transformers have creat...,2023-06-09 15:44:54
544,Zidong Taichu,Multimodal,NaN,Chinese Academy of Sciences,NaN,NaN,NaN,Zidong Ancestral multi-modal large model,https://gitee.com/zidongtaichu/multi-modal-models,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Likely,NaN,2023-06-09 15:44:36
546,NaN,Vision,Image classification,"University of Guelph,Canadian Institute for Ad...",Industry - Academia Collaboration,"Terrance DeVries, Graham W. Taylor",2017-08-15,Improved Regularization of Convolutional Neura...,https://arxiv.org/abs/1708.04552,1450.0,...,NaN,NaN,NaN,https://www.yuzeh.com/data/agz-cost.html,NaN,NaN,Industry,NaN,NaN,2023-06-09 16:00:52


In [20]:
# Count the number of notable ML systems for each Organization since 2010.
organization_system_count = defaultdict(int)
for i, row in notable_df.iterrows():
    pub_date = row['Publication date']
    if type(pub_date) == str and int(pub_date[:4]) >= 2010 and row['Organization Categorization'] == 'Industry':
        org = row['Organization']
        organization_system_count[org] += 1

In [21]:
# Print organization and its system count, in descending order of count
for org, count in sorted(organization_system_count.items(), key=lambda x: x[1], reverse=True):
    print(f"{org}: {count} systems")

Google: 20 systems
DeepMind: 16 systems
OpenAI: 12 systems
Google Research: 7 systems
Google Brain: 7 systems
Google DeepMind: 7 systems
Microsoft Research: 5 systems
Meta AI: 4 systems
Facebook AI Research: 4 systems
Facebook AI research: 4 systems
MetaAI: 3 systems
Microsoft: 3 systems
Alibaba Group: 2 systems
Facebook: 2 systems
Facebook AI: 2 systems
Google Inc.: 2 systems
Amazon: 1 systems
Stability AI, Runway: 1 systems
Google AI, Brain team: 1 systems
Microsoft Research,Peking University: 1 systems
Open AI: 1 systems
Microsoft Bing: 1 systems
Google Research, Brain Team: 1 systems
Google Research,Brain Team: 1 systems
Google AI: 1 systems
AllenAI, University of Washington: 1 systems
Google Brain,Google Research: 1 systems
Twitter: 1 systems
Megvii Inc: 1 systems
Nvidia: 1 systems
Salesforce: 1 systems
Baidu Research- Silicon Valley AI Lab: 1 systems
Netflix: 1 systems
Xerox Research Centre Europe (XRCE): 1 systems
Google Inc: 1 systems
NVIDIA: 1 systems
Baidu: 1 systems


Industry:
1. Google: 42
  - Google: 20
  - Google Research: 7
  - Google Brain: 7
  - Google Inc.: 2
  - Google AI, Brain team: 1
  - Google Research, Brain Team: 1
  - Google Research,Brain Team: 1
  - Google AI: 1
  - Google Brain,Google Research: 1
  - Google Inc: 1
2. DeepMind: 23
  - DeepMind: 16
  - Google DeepMind: 7
3. Meta: 20
  - Meta AI: 4
  - MetaAI: 4
  - Facebook AI Research: 4
  - Facebook AI research: 4
  - Facebook: 2
  - Facebook AI 2
4. OpenAI: 13
   1. OpenAI: 12
   2. Open AI: 1
5. Microsoft: 10
   - Microsoft Research: 5
   - Microsoft: 3
   - Microsoft Research,Peking University: 1
   - Microsoft Bing: 1
6. Alibaba: 2
  - Alibaba Group: 2
6. NVIDIA: 2
  - Nvidia: 1
  - NVIDIA: 1
6. Baidu: 2
  - Baidu Research- Silicon Valley AI Lab: 1
  - Baidu: 1
7. Amazon: 1
  - Amazon: 1
7. Stability: 1
  - Stability AI, Runway: 1
7. Runway: 1
  - Stability AI, Runway: 1
7. Twitter: 1
7. Megvii: 
  - Megvii Inc: 1
7. Salesforce: 1
  - Salesforce: 1
7. Netflix: 1
  - Netflix: 1
7. Xerox: 1
  - Xerox Research Centre Europe (XRCE): 1

# OpenAlex

In [63]:
concept_ids = [
    # 'https://openalex.org/C154945302',  # Artificial intelligence
    'https://openalex.org/C119857082',  # Machine learning
]

In [64]:
concept_query = "|".join(concept_ids)
concept_query

'https://openalex.org/C119857082'

In [66]:
top_institutions = merge_pages(
    Institutions() \
        .filter(concepts={"id": concept_query}) \
        .filter(type="company") \
        .sort(works_count="desc") \
        .paginate(per_page=200, n_max=200)
)

In [67]:
top_institutions[1]['counts_by_year']

[{'year': 2023, 'works_count': 432, 'cited_by_count': 254671},
 {'year': 2022, 'works_count': 1238, 'cited_by_count': 594200},
 {'year': 2021, 'works_count': 3008, 'cited_by_count': 1006528},
 {'year': 2020, 'works_count': 3192, 'cited_by_count': 827724},
 {'year': 2019, 'works_count': 2818, 'cited_by_count': 557240},
 {'year': 2018, 'works_count': 2073, 'cited_by_count': 326302},
 {'year': 2017, 'works_count': 1536, 'cited_by_count': 176882},
 {'year': 2016, 'works_count': 1265, 'cited_by_count': 121938},
 {'year': 2015, 'works_count': 1105, 'cited_by_count': 87186},
 {'year': 2014, 'works_count': 892, 'cited_by_count': 61367},
 {'year': 2013, 'works_count': 833, 'cited_by_count': 45477},
 {'year': 2012, 'works_count': 691, 'cited_by_count': 33851}]

In [68]:
ins_cited_by_count = defaultdict(int)
ins_works_count = defaultdict(int)
for ins_obj in top_institutions:
    counts_by_year = ins_obj['counts_by_year']
    for year_counts in counts_by_year:
        if year_counts['year'] >= 2010:
            ins_cited_by_count[ins_obj['id']] += year_counts['cited_by_count']
            ins_works_count[ins_obj['id']] += year_counts['works_count']

In [69]:
for i, (ins, works_count) in enumerate(sorted(ins_works_count.items(), key=lambda x: x[1], reverse=True)):
    if i > 20:
        break
    print(f"{Institutions()[ins]['display_name']}: {works_count} works")

Google (United States): 19083 works
Microsoft (United States): 15487 works
Microsoft Research (United Kingdom): 7212 works
Huawei Technologies (China): 7017 works
Amazon (United States): 5459 works
Tencent (China): 4535 works
Meta (Israel): 4244 works
Tata Consultancy Services (India): 3574 works
China Mobile (China): 3467 works
Alibaba Group (China): 3428 works
Adobe Systems (United States): 3415 works
Microsoft Research Asia (China): 3374 works
Baidu (China): 2823 works
Decision Systems (United States): 2765 works
Aditya Birla (India): 2529 works
Management Sciences (United States): 2309 works
Meta (United States): 2173 works
Amazon (Germany): 2104 works
Aselsan (Turkey): 1872 works
Nvidia (United States): 1782 works
Alibaba Group (United States): 1675 works
